# Train your first Deep Reinforcement Learning Agent

PYTHON 3.9

El objetivo de este notebook es crear un sistema para entrenar los algoritmos que sea claro y escalable. Así que voy a tratar el menor número de celdas posibles

In [1]:
import sys #-- lo he necesitado para importar el paquete
sys.path.append('..')
import gym_hearthstone

## Argumentos
He añadido argumentos al environment para que sea más sencillo hacer el entrenamiento:

    action_type: Define el espacio de acción que se va a crear. Ahora mismo solo hay dos implementados.
    
        * "random"  -> Solo una acción possible (Tomar una acción aleatoria de las que son posibles).
        * "type_rd" -> 6 acciones posibles (Escoge un tipo de accion sino es legal hace una de cualquier tipo).
        * "type"    -> 6 acciones posibles (Toma una aleatoria según el tipo de acción y sino es legal no hace nada).
        
        
        
    reward_mode: Define el reward que devuelve el environment:
    
        * "simple"    -> Solo hay reward cuando se gana o se pierde.
        * "penalize"  -> Se penalizan las acciones que no son válidas.
        * "incentive" -> Se premian las acciones que son válidas.
        * "complex"   -> Se penalizan las acciones que no son válidas y se premian las válidas.


    steps: Por ahora usa el numero de steps para el nombre del fichero de salida. 
    
        
He comprobado que funciona ejecutando los steps manualmente, pero todavía me queda comprobar el dqn en el otro ordenador. Además, todavía no he implementado que no ejecute ninguna acción cuándo la acción elegida no es válida (ahora mismo penaliza y hace otra acción aleatoria en ese step).

In [2]:
import gymnasium as gym
import gym_hearthstone
from stable_baselines3 import DQN
import time
import os

In [3]:

steps = 100
env = gym.make('Hearthstone-v1', action_type = "type", reward_mode = "complex", steps = steps)

[fireplace.__init__]: Initializing card database
[fireplace.__init__]: Merged 26517 cards
[fireplace.entity]: Setting up game Game(players=(Player(name='Player1', hero=None), Player(name='Player2', hero=None)))
[fireplace.entity]: Tossing the coin... Player2 wins!
[fireplace.actions]: Player(name='Player1', hero=<Hero ('Rexxar')>) triggering <TargetedAction: Summon(<Summon.CARD>=<HeroPower ('Steady Shot')>)> targeting [Player(name='Player1', hero=<Hero ('Rexxar')>)]
[fireplace.actions]: Player1 summons [<HeroPower ('Steady Shot')>]
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.actions]: Player(name='Player1', hero=<Hero ('Rexxar')>) triggering <TargetedAction: Summon(<Summon.CARD>=<Hero ('Rexxar')>)> targeting [Player(name='Player1', hero=<Hero ('Rexxar')>)]
[fireplace.actions]: Player1 summons [<Hero ('Rexxar')>]
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.entity]: Player(name='Player1', hero=<Hero ('Rexxar')>

IMPLEMENTED CARDS: 8022
UNIMPLEMENTED CARDS: 18495
Implemented but Uncollectible: 6119
CardClass.DEATHKNIGHT amount of cards is: 0
CardClass.DRUID amount of cards is: 130
CardClass.HUNTER amount of cards is: 132
CardClass.MAGE amount of cards is: 128
CardClass.PALADIN amount of cards is: 128
CardClass.PRIEST amount of cards is: 119
CardClass.ROGUE amount of cards is: 123
CardClass.SHAMAN amount of cards is: 126
CardClass.WARLOCK amount of cards is: 118
CardClass.WARRIOR amount of cards is: 120
CardClass.DEMONHUNTER amount of cards is: 41
CardClass.NEUTRAL amount of cards is: 738
HearthstoneEnv - Version 0.2.0
Step.BEGIN_MULLIGAN
Step.MAIN_ACTION


In [4]:
print(os.getpid())


model = DQN("MultiInputPolicy", env, verbose=1)
env.reset_stats()
start = time.time()
env.reset()

model.learn(total_timesteps=steps, log_interval=4)

end = time.time()
env.display_stats()

total_time = end-start

if(env.curr_episode > 0):
    avg_time = (end-start) / env.curr_episode
    avg_steps = steps / env.curr_episode
else:
        avg_time = (end-start) 
        avg_steps = steps 


if (env.wins + env.losses > 0):
    win_rate = env.wins / (env.wins + env.losses)
else:
    win_rate = 0

print("Avg steps per game: " + str(avg_steps))
print("Total running time: {:.2f} and running time per game: {:.2f}".format(total_time, avg_time) )

data_file = open('./metrics/' + env.file_name + '.txt', 'w')
data_file.write("Games, AvgStepsGame, TotalTime, AvgTimeGame, TotalReward, WinRate \n")
data_file.write("{},{:.2f},{:.2f},{:.2f},{:.2f},{:.2f}".format(env.curr_episode, avg_steps, total_time, avg_time, env.total_reward, win_rate))
data_file.close()

model.save("./dqn_models/" + env.file_name + "_dqn")
env.close()




C:\Users\jesus\anaconda3\envs\env_gym-hearthstone\lib\site-packages\stable_baselines3\common\buffers.py:564: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 5.84GB > 0.21GB
  warnings.warn(


10304
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


C:\Users\jesus\anaconda3\envs\env_gym-hearthstone\lib\site-packages\gymnasium\utils\passive_env_checker.py:197: DeprecationWarning: WARN: Current gymnasium version requires that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.deprecation(
C:\Users\jesus\anaconda3\envs\env_gym-hearthstone\lib\site-packages\gymnasium\utils\passive_env_checker.py:210: DeprecationWarning: WARN: Current gymnasium version requires that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.deprecation(
[fireplace.entity]: Setting up game Game(players=(Player(name='Player1', hero=None), Player(name='Player2', hero=None)))
[fireplace.entity]: Tossing the coin... Player1 wins!
[fireplace.actions]: Player(name='Player1', hero=<Hero ('Rexxar')>) triggering <TargetedAction: Summon(<Summon.CARD>=<HeroPower ('Steady Shot')>)> targeting [Player(name='Player1', hero=<Hero ('

Step.BEGIN_MULLIGAN
Step.MAIN_ACTION
1630
1612
4778
5680
184
2843
2953
2828
3031
Step.BEGIN_MULLIGAN


[fireplace.entity]: Player2 is now at 1 mana crystals
[fireplace.actions]: Player(name='Player2', hero=<Hero ('Garrosh Hellscream')>) triggering <TargetedAction: Draw()> targeting [Player(name='Player2', hero=<Hero ('Garrosh Hellscream')>)]
[fireplace.entity]: Player2 draws <Minion ('Hungry Crab')>
[fireplace.card]: <Minion ('Hungry Crab')> moves from <Zone.DECK: 2> to <Zone.HAND: 3>
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.actions]: Player(name='Player1', hero=<Hero ('Rexxar')>) triggering <TargetedAction: Draw()> targeting [Player(name='Player1', hero=<Hero ('Rexxar')>)]
[fireplace.entity]: Player1 draws <Minion ('Zixor, Apex Predator')>
[fireplace.card]: <Minion ('Zixor, Apex Predator')> moves from <Zone.DECK: 2> to <Zone.HAND: 3>
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.card]: <Minion ('Corpse Widow')> moves from <Zone.HAND: 3> to <Zone.DECK: 2>
[fireplace.entity]: Player(name='Player1', hero=<Hero 

...

[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.entity]: Player2 ends turn 8
[fireplace.entity]: Player1 begins turn 9
[fireplace.entity]: Player1 is now at 4 mana crystals
[fireplace.actions]: Player(name='Player1', hero=<Hero ('Rexxar')>) triggering <TargetedAction: Draw()> targeting [Player(name='Player1', hero=<Hero ('Rexxar')>)]
[fireplace.entity]: Player1 draws <Minion ('Twilight Summoner')>
[fireplace.card]: <Minion ('Twilight Summoner')> moves from <Zone.DECK: 2> to <Zone.HAND: 3>
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.actions]: Player1 plays <Minion ('Ironforge Rifleman')> (target=<Hero ('Garrosh Hellscream')>, index=None)
[fireplace.entity]: Player1 pays 3 mana
[fireplace.card]: <Minion ('Ironforge Rifleman')> moves from <Zone.HAND: 3> to <Zone.PLAY: 1>
[fireplace.actions]: <Minion ('Ironforge Rifleman')> triggering <TargetedAction: Battlecry(<Battlecry.TARGET>=<Hero ('Garrosh Hellscream')>)> targe

2969
7873
3220
7967
2459
840
5689
5689
3799
1967
2962
1603
7861
------------------------
>>> Current Step 14
------------------------
{'choice': [], 'play_card': [[<Move.play_card: 4>, 0, None, None], [<Move.play_card: 4>, 2, None, None], [<Move.play_card: 4>, 3, None, None], [<Move.play_card: 4>, 4, None, None], [<Move.play_card: 4>, 5, None, None], [<Move.play_card: 4>, 6, None, None], [<Move.play_card: 4>, 7, None, None]], 'heropower': [[<Move.hero_power: 1>, None, None]], 'minion_attack': [], 'hero_attack': [], 'end_turn': [[<Move.end_turn: 0>]]}

>>> possible_actions 9:[[<Move.play_card: 4>, 0, None, None], [<Move.play_card: 4>, 2, None, None], [<Move.play_card: 4>, 3, None, None], [<Move.play_card: 4>, 4, None, None], [<Move.play_card: 4>, 5, None, None], [<Move.play_card: 4>, 6, None, None], [<Move.play_card: 4>, 7, None, None], [<Move.hero_power: 1>, None, None], [<Move.end_turn: 0>]]
>>> alreadySelectedActions 0:[]
>>> possible_actions 9:[[<Move.play_card: 4>, 0, None, None], 

[fireplace.actions]: Activating <Minion ('Ironforge Rifleman')> action targeting <Hero ('Garrosh Hellscream')>
[fireplace.actions]: <Minion ('Ironforge Rifleman')> triggering <TargetedAction: Hit(<Hit.AMOUNT>=1)> targeting [<Hero ('Garrosh Hellscream')>]
[fireplace.actions]: <Minion ('Ironforge Rifleman')> triggering <TargetedAction: Predamage(<Predamage.AMOUNT>=1)> targeting [<Hero ('Garrosh Hellscream')>]
[fireplace.actions]: <Minion ('Ironforge Rifleman')> triggering <TargetedAction: Damage()> targeting [<Hero ('Garrosh Hellscream')>]
[fireplace.entity]: <Hero ('Garrosh Hellscream')> loses 1 armor instead of damage
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.actions]: <Minion ('Crackling Razormaw')> attacks <Hero ('Garrosh Hellscream')>
[fireplace.actions]: <Minion ('Crackling Razormaw')> triggering <TargetedAction: Hit(<Hit.AMOUNT>=3)> targeting [<Hero ('Garrosh Hellscream

{'choice': [], 'play_card': [], 'heropower': [], 'minion_attack': [[<Move.minion_attack: 2>, 0, 0]], 'hero_attack': [], 'end_turn': [[<Move.end_turn: 0>]]}
>>> possible_actions 2:[[<Move.minion_attack: 2>, 0, 0], [<Move.end_turn: 0>]]
>>> alreadySelectedActions 0:[]
doing single turn for rando
Doing action: [<Move.minion_attack: 2>, 0, 0]
{'choice': [], 'play_card': [], 'heropower': [], 'minion_attack': [], 'hero_attack': [], 'end_turn': [[<Move.end_turn: 0>]]}
>>> possible_actions 1:[[<Move.end_turn: 0>]]
>>> alreadySelectedActions 0:[]

doing end turn for rando
Doing action: [<Move.end_turn: 0>]
7938
2969
7873
3220
7967
2459
840
5689
5689
5656
3799
2962
1603
1967
7861
------------------------
>>> Current Step 15
------------------------
{'choice': [], 'play_card': [[<Move.play_card: 4>, 0, None, None], [<Move.play_card: 4>, 2, None, None], [<Move.play_card: 4>, 3, None, None], [<Move.play_card: 4>, 4, None, None], [<Move.play_card: 4>, 5, None, None], [<Move.play_card: 4>, 6, None, N

C:\Users\jesus\anaconda3\envs\env_gym-hearthstone\lib\site-packages\stable_baselines3\common\save_util.py:278: UserWarning: Path 'dqn_models' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")
